In [529]:
import numpy as np
import pandas as pd
from datascience import *



In [530]:
LDLR_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LDLR_table_refseq.txt'
SREBF2_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/SREBF2_table_refseq.txt'
HNF4A_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/HNF4A_table_refseq.txt'
MYLIP_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/MYLIP_table_refseq.txt'
ACAT2_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/ACAT2_table_refseq.txt'
LSS_file_path = '/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LSS_table_refseq.txt'


LDLR_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/LDLR_sequence_path.txt"
SREBF2_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/SREBF2_sequence_path.txt"
HNF4A_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/HNF4A_sequence_path.txt"
MYLIP_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/MYLIP_sequence_path.txt"
ACAT2_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/ACAT2_sequence_path.txt"
LSS_sequence_path = "/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/sequence_files/LSS_sequence_path2.txt"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [531]:
np.arange(5,-1,-1)

array([5, 4, 3, 2, 1, 0])

In [532]:
def sequence_table(gene, sequence_file_path, gene_file_path):
    """returns table with the sequence and reading frame of each exon. Inputs are exon sequence file and RefGene file from UCSC table browser."""
    seq_file = open(sequence_file_path).read()
    genes = seq_file.split(">")
    del genes[0]
    gene = []
    sequence = []
    for num in np.arange(len(genes)-1,-1,-1):
        gene = np.append(gene, genes[num].split("=none\n")[0])
        sequence = np.append(sequence, genes[num].split("=none\n")[1].replace("\n", ""))
    print(len(gene), len(sequence))
    gene_file = pd.read_csv(gene_file_path, sep = "\t").loc[0]
    reading_frames = gene_file.loc["exonFrames"].strip(",").split(",")
    exon_starts = gene_file.loc["exonStarts"].strip(",").split(",")
    exon_ends = gene_file.loc["exonEnds"].strip(",").split(",")
    print(len(exon_ends))
    if reading_frames[0]=='-1':
        del reading_frames[0]
        del exon_starts[0]
        del exon_ends[0]
    seq_table = pd.DataFrame({"GeneID": gene, "chromosome": gene_file.loc["chrom"].strip("chr"), "strand": gene_file.loc["strand"], "sequence": sequence, "reading_frame":reading_frames, "exon_start": exon_starts, "exon_end": exon_ends})
    return seq_table

In [533]:
def stop_gain_targets(gene, sequence_file_path, gene_file_path):
    seq_table = sequence_table(gene, sequence_file_path, gene_file_path).drop(0)
    rv_target_codons = ["TGG", "CGA"]
    fw_target_codons = ["CGA", "CAA", "CAG"]
    guide1s = []
    guide2s = []
    guide3s = []
    guide4s = []
    guide5s = []
    target_strand = []
    target_codons = []
    target_exon = []
    reading_frames = []
    for i in np.arange(1, len(seq_table)+1):
        row = seq_table.loc[i]
        exon = i+1
        reading_frame = int(row.loc["reading_frame"])
        sequence = row.loc["sequence"]
        if reading_frame == -1:
            in_frame_sequence = sequence[2:len(sequence)]
            adjust = 3
        elif reading_frame == 0:
            in_frame_sequence = sequence[reading_frame:len(sequence)+1]
            adjust = -8
        elif reading_frame == 1:
            in_frame_sequence = sequence[reading_frame:len(sequence)+1]
            adjust = -4
        else:
            in_frame_sequence = sequence[reading_frame:len(sequence)+1]
            adjust = 0
        codons = []
        for i in np.arange(0,len(in_frame_sequence), 3):
            codon = in_frame_sequence[i+reading_frame:(i+reading_frame+3)]
            codons = np.append(codons, codon)
            if codon in fw_target_codons:
                target_base = int(row.loc["exon_end"]) - adjust +(2*reading_frame) -i -22
                guide_1_start = target_base-2
                guide_1_end = guide_1_start+19
                guide_2_start = guide_1_start + 1
                guide_2_end = guide_2_start + 19
                guide_3_start = guide_1_start + 2
                guide_3_end = guide_3_start + 19
                guide_4_start = guide_1_start + 3
                guide_4_end = guide_4_start + 19
                guide_5_start = guide_1_start + 4
                guide_5_end = guide_5_start + 19
                target_strand = np.append(target_strand, row.loc["strand"])
                guide1s = np.append(guide1s, "chr"+str(row.loc["chromosome"])+":"+str(guide_1_start)+"-"+str(guide_1_end))
                guide2s = np.append(guide2s, "chr"+str(row.loc["chromosome"])+":"+str(guide_2_start)+"-"+str(guide_2_end))
                guide3s = np.append(guide3s, "chr"+str(row.loc["chromosome"])+":"+str(guide_3_start)+"-"+str(guide_3_end))
                guide4s = np.append(guide4s, "chr"+str(row.loc["chromosome"])+":"+str(guide_4_start)+"-"+str(guide_4_end))
                guide5s = np.append(guide5s, "chr"+str(row.loc["chromosome"])+":"+str(guide_5_start)+"-"+str(guide_5_end))
                target_codons = np.append(target_codons, codon)
                target_exon = np.append(target_exon, exon)
                reading_frames = np.append(reading_frames, reading_frame)
            if codon in rv_target_codons:
                target_base = int(row.loc["exon_end"]) - adjust + reading_frame -i +2
                guide_1_end = target_base + reading_frame + 5
                guide_1_start = guide_1_end-19
                guide_2_start = guide_1_start -1
                guide_2_end = guide_1_end - 1
                guide_3_start = guide_1_start - 2
                guide_3_end = guide_1_end - 2
                guide_4_start = guide_1_start - 3
                guide_4_end = guide_1_end - 3
                guide_5_start = guide_1_start - 4
                guide_5_end = guide_1_end - 4
                target_codons = np.append(target_codons, codon)
                target_exon = np.append(target_exon, exon)
                reading_frames = np.append(reading_frames, reading_frame)
                if row.loc["strand"] == "+":
                    target_strand = np.append(target_strand, "-")
                else:
                    target_strand = np.append(target_strand, "+")
                guide1s = np.append(guide1s, "chr"+str(row.loc["chromosome"])+":"+str(guide_1_start)+"-"+str(guide_1_end))
                guide2s = np.append(guide2s, "chr"+str(row.loc["chromosome"])+":"+str(guide_2_start)+"-"+str(guide_2_end))
                guide3s = np.append(guide3s, "chr"+str(row.loc["chromosome"])+":"+str(guide_3_start)+"-"+str(guide_3_end))
                guide4s = np.append(guide4s, "chr"+str(row.loc["chromosome"])+":"+str(guide_4_start)+"-"+str(guide_4_end))
                guide5s = np.append(guide5s, "chr"+str(row.loc["chromosome"])+":"+str(guide_5_start)+"-"+str(guide_5_end))
            else:
                continue

    target_table = pd.DataFrame({"gene": gene, "site": "stop/gain", "exon":target_exon, "strand": target_strand, "BE": "CBE", "target_codon": target_codons, "reading_frame": reading_frames, "guide1":guide1s , "guide2":guide2s, "guide3":guide3s, "guide4":guide4s, "guide5":guide5s})
    return target_table[target_table["exon"]!=1]
        

In [534]:
LSS = stop_gain_targets("LSS", LSS_sequence_path, LSS_file_path)

22 22
23


In [535]:
seqq= open(LSS_sequence_path).read().split(">")
del seqq[0]
seqq[20].split("=none\n")[0]

"hg38_refGene_NM_001001438_20 range=chr21:46191881-46191959 5'pad=0 3'pad=0 strand=- repeatMasking"

In [536]:
a = pd.read_csv(LSS_file_path, sep="\t")

In [537]:
a

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,937,NM_001001438,chr21,-,46188445,46228824,46191103,46228745,23,"46188445,46191065,46191880,46194490,46195675,4...","46188821,46191235,46191959,46194661,46195756,4...",0,LSS,cmpl,cmpl,"-1,0,2,2,2,2,1,0,0,0,0,0,2,0,1,0,2,1,2,1,0,2,0,"
1,937,NM_001145436,chr21,-,46188445,46228774,46191103,46228745,22,"46188445,46191880,46194490,46195675,46196201,4...","46191235,46191959,46194661,46195756,46196267,4...",0,LSS,cmpl,cmpl,"0,2,2,2,2,1,0,0,0,0,0,2,0,1,0,2,1,2,1,0,2,0,"
2,937,NM_002340,chr21,-,46188445,46228774,46191103,46228745,22,"46188445,46191880,46194490,46195675,46196201,4...","46191235,46191959,46194661,46195756,46196267,4...",0,LSS,cmpl,cmpl,"0,2,2,2,2,1,0,0,0,0,0,2,0,1,0,2,1,2,1,0,2,0,"
3,937,NM_001145437,chr21,-,46189123,46228824,46191103,46227630,21,"46189123,46191880,46194490,46195675,46196201,4...","46191235,46191959,46194661,46195756,46196267,4...",0,LSS,cmpl,cmpl,"0,2,2,2,2,1,0,0,0,0,0,2,0,1,0,2,1,2,1,0,-1,"


In [538]:
LSS

,gene,site,exon,strand,BE,target_codon,reading_frame,guide1,guide2,guide3,guide4,guide5
0,LSS,stop/gain,2.0,-,CBE,CAG,2.0,chr21:46191924-46191943,chr21:46191925-46191944,chr21:46191926-46191945,chr21:46191927-46191946,chr21:46191928-46191947
1,LSS,stop/gain,2.0,-,CBE,CAG,2.0,chr21:46191891-46191910,chr21:46191892-46191911,chr21:46191893-46191912,chr21:46191894-46191913,chr21:46191895-46191914
2,LSS,stop/gain,2.0,+,CBE,TGG,2.0,chr21:46191885-46191904,chr21:46191884-46191903,chr21:46191883-46191902,chr21:46191882-46191901,chr21:46191881-46191900
3,LSS,stop/gain,2.0,-,CBE,CAG,2.0,chr21:46191867-46191886,chr21:46191868-46191887,chr21:46191869-46191888,chr21:46191870-46191889,chr21:46191871-46191890
4,LSS,stop/gain,3.0,-,CBE,CAG,2.0,chr21:46194596-46194615,chr21:46194597-46194616,chr21:46194598-46194617,chr21:46194599-46194618,chr21:46194600-46194619
5,LSS,stop/gain,3.0,+,CBE,TGG,2.0,chr21:46194590-46194609,chr21:46194589-46194608,chr21:46194588-46194607,chr21:46194587-46194606,chr21:46194586-46194605
6,LSS,stop/gain,3.0,-,CBE,CAG,2.0,chr21:46194536-46194555,chr21:46194537-46194556,chr21:46194538-46194557,chr21:46194539-46194558,chr21:46194540-46194559
7,LSS,stop/gain,3.0,-,CBE,CAG,2.0,chr21:46194527-46194546,chr21:46194528-46194547,chr21:46194529-46194548,chr21:46194530-46194549,chr21:46194531-46194550
8,LSS,stop/gain,3.0,-,CBE,CAG,2.0,chr21:46194521-46194540,chr21:46194522-46194541,chr21:46194523-46194542,chr21:46194524-46194543,chr21:46194525-46194544
9,LSS,stop/gain,3.0,+,CBE,TGG,2.0,chr21:46194515-46194534,chr21:46194514-46194533,chr21:46194513-46194532,chr21:46194512-46194531,chr21:46194511-46194530


In [540]:
#LSS.to_csv('/Users/allisonjames/Documents/Rotations/Sherwood Lab/BE screens/LSS_CBE_controls.csv')